# HMS Capstone - Machine Learning - Random Forests
## By: AJ Goldstein (https://github.com/ajva1996)

### <span style="color:red">Modeling Task: </span>
### Understand the relationship between psychological inflexibility (i.e. AAQ) and mental health outcomes (i.e. depression, anxiety, well-being) while controlling for key demographic info (i.e. race, gender, field of study)

In [573]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
sns.set_context("talk")

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## STEP #0: Carry forward data from previous notebook

In [574]:
# cleaned dataset
%store -r HMS

# separated modules
%store -r HMS_ids
%store -r HMS_demo
%store -r HMS_mhstatus
%store -r HMS_mhhelp
%store -r HMS_aaq

# tidy variable groups
%store -r tidy_race
%store -r tidy_religion
%store -r tidy_degreeType
%store -r tidy_fieldOfStudy
%store -r tidy_activity
%store -r tidy_age
%store -r tidy_gender
%store -r tidy_relig

## STEP #1: Prepare data for classification

### a) convert demographics & AAQ into usable features matrix (X)

In [575]:
# select columns without text 
cols = [col for col in HMS_demo.columns if ('text' not in col)]
X = HMS_demo[cols].fillna(0)

In [576]:
# convert them to categorical variables
for col in cols:
    X[col] = X[col].astype('category')

In [577]:
# add in AAQ_total score
X.insert(loc=0, column = 'AAQ', value = HMS_aaq['AAQ_total'])
feat_cols = X.columns

X.head()

,AAQ,age,sex_birth,gender,sexual,relship,race_black,race_ainaan,race_asian,race_his_temp,...,disab_1_1,disab_1_2,disab_1_3,disab_1_4,disab_1_5,disab_1_6,disab_1_7,disab_1_8,disab_1_9,disab_3
0,NaN,20.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,21.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,NaN,22.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,19.0,2.0,1.0,1.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,NaN,21.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### b) remove unapplicable and missing values (NaN's)

In [578]:
# select three main outcomes
Y_flourish = HMS_mhstatus['flourish']
Y_depression = HMS_mhstatus['deprawsc']
Y_anxiety = HMS_mhstatus['anx_score']

In [579]:
# combine all variables (X's and Y's) into one dataframe
all_variables = pd.concat([X, Y_flourish, Y_depression, Y_anxiety], axis = 1)

In [580]:
# remove all rows with any NaN
print('shape before removing NaNs:', all_variables.shape)
all_variables = all_variables.dropna(axis=0, how='any')
print('shape after removing NaNs:', all_variables.shape) # NOTE: 25,000+ dropped rows were due to AAQ

('shape before removing NaNs:', (45756, 103))
('shape after removing NaNs:', (22781, 103))


#### <span style="color:red"> IMPORTANT NOTE: 25,000+ dropped rows were due to AAQ (half of respondents did not have a score)</span>

In [581]:
# remove middle 1/3 of scores in the "mild" range
print('shape BEFORE removing middle 1/3 of scores:', all_variables.shape)
all_variables = all_variables.loc[~all_variables['flourish'].between(42, 47, inclusive=True)]
all_variables = all_variables.loc[~all_variables['deprawsc'].between(5, 9, inclusive=True)]
all_variables = all_variables.loc[~all_variables['anx_score'].between(5, 9, inclusive=True)]
print('shape AFTER removing middle 1/3 of scores:', all_variables.shape)

('shape BEFORE removing middle 1/3 of scores:', (22781, 103))
('shape AFTER removing middle 1/3 of scores:', (8972, 103))


#### <span style="color:red"> IMPORTANT NOTE: 13,000 rows were dropped from the "mild" range of these scores</span>

In [582]:
# split big dataframe back up
X = all_variables.drop(labels=['flourish', 'deprawsc', 'anx_score'], axis=1)
Y_flourish_cleaned = all_variables['flourish']
Y_depression_cleaned = all_variables['deprawsc']
Y_anxiety_cleaned = all_variables['anx_score']

### b) categorize mental health outcomes into classified labels

In [583]:
def categorize_outcome(value, outcome):
    
    # flourishing scale
    if outcome == 'flourish':
        if pd.isnull(value):
            return value
        elif value < 48:
            return 0
        elif value >= 48:
            return 1
        
    # depression or anxiety
    else:
        if pd.isnull(value):
            return value
        elif value < 10:
            return 0
        elif value >= 10:
            return 1

In [584]:
# convert to categorical labels
Y_flourish_cats = pd.Series([categorize_outcome(score, 'flourish') for score in Y_flourish_cleaned], name='flourish')
Y_depression_cats = pd.Series([categorize_outcome(score, 'depression') for score in Y_depression_cleaned], name = 'depression')
Y_anxiety_cats = pd.Series([categorize_outcome(score, 'anxiety') for score in Y_anxiety_cleaned], name = 'anxiety')

#### <span style="color:red">NOTE: using "moderate" score of 10 as cutoff for depression/anxiety... 48 as the threshold for positive mental health</span>

## STEP #2: Train a Random Forest Classifier

### split data into train & test sets

In [585]:
# split the data into training and test sets
X_train, X_test = train_test_split(X, test_size=0.3, random_state=0)
y_flourish_train, y_flourish_test = train_test_split(Y_flourish_cats, test_size=0.3, random_state=0)
y_depression_train, y_depression_test = train_test_split(Y_depression_cats, test_size=0.3, random_state=0)
y_anxiety_train, y_anxiety_test = train_test_split(Y_anxiety_cats, test_size=0.3, random_state=0)

### create and train the classifier

In [586]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=1000, max_features = "auto", max_depth = 5,
                             min_samples_leaf = 100, random_state=0, n_jobs=-1)

In [587]:
# Train the classifier on all three outcomes
clf.fit(X_train, y_depression_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=100, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

### extract feature importances

In [588]:
# Print the name and gini importance of each feature
features = pd.DataFrame(sorted(zip(feat_cols, clf.feature_importances_),
                               key=lambda x: x[1], reverse=True), columns = ['name', 'importance'])
features.head(10)

,name,importance
0,AAQ,0.366570
1,aca_impa,0.269347
2,persist,0.106480
3,fincur,0.093753
4,sexual,0.044024
5,finpast,0.031884
6,gpa_sr,0.017441
7,religios,0.010687
8,gender,0.009129
9,relig_aff_ch,0.006497


### Test the accuracy of our full-feature classifier

In [589]:
# Apply The Full Featured Classifier To The Test Data
y_pred = clf.predict(X_test)

# View The Accuracy Of Our Full Feature (4 Features) Model
accuracy_score(y_depression_test, y_pred)

0.90973254086181277

### Optimize hyperparameters using GridSearch

In [590]:
from sklearn.grid_search import GridSearchCV
 
rfc = RandomForestClassifier(n_estimators = 1000, n_jobs=-1, oob_score = True) 
 
# Use a grid over parameters of interest
param_grid = {"criterion" : ['gini', 'entropy'],
              "max_depth" : [3, 4, 5, 6],
              "max_features" : [4, 6, 8, 10],
              "min_samples_leaf" : [10, 20, 30, 50]}
 
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(X_train, y_depression_train)
print CV_rfc.best_params_

KeyboardInterrupt: 

In [ ]:
"n_estimators" : [10, 100, 500, 1000, 10000, 100000],
           "max_features" : [1, 2, 4, 8, 16, 32],
           "max_depth" : [1, 5, 10, 15, 20, 25, 30],
            "min_samples_leaf" : [1, 2, 4, 6, 8, 10]

### Identify and select the most important features

In [591]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0
sfm = SelectFromModel(clf, threshold=0.03)

# Train the selector
sfm.fit(X_train, y_depression_train)

# Print the names of the most important features
feat_cols_important = []
for feature_list_index in sfm.get_support(indices=True):
    feat_cols_important.append(feat_cols[feature_list_index])
    print(feat_cols[feature_list_index])

AAQ
sexual
fincur
finpast
aca_impa
persist


### Create a dataset with only the most important features

In [592]:
# Transform the data to create a new dataset containing only the most important features
# Note: We have to apply the transform to both the training X and test X data.
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)

### Train a new random forest classifier using only the most important features

In [593]:
# Create a new random forest classifier for the most important features
clf = RandomForestClassifier(n_estimators=1000, max_features = "auto", max_depth = 5,
                             min_samples_leaf = 10, random_state=0, n_jobs=-1)

# Train the new classifier on the new dataset containing the most important features
clf_important.fit(X_important_train, y_depression_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

### extract feature importances

In [594]:
# Print the name and gini importance of each feature
features = pd.DataFrame(sorted(zip(feat_cols_important, clf_important.feature_importances_),
                               key=lambda x: x[1], reverse=True), columns = ['name', 'importance'])
features.head(15)

,name,importance
0,AAQ,0.505255
1,aca_impa,0.265576
2,fincur,0.077233
3,persist,0.072019
4,finpast,0.046874
5,sexual,0.033042


### Test the accuracy of our limited-feature model

In [595]:
# Apply The Full Featured Classifier To The Test Data
y_important_pred = clf_important.predict(X_important_test)

# View The Accuracy Of Our Limited Feature (2 Features) Model
accuracy_score(y_depression_test, y_important_pred)

0.90378900445765231

## STEP #3: Train a Random Forest Regressor

### 1) Quick cleaning

In [596]:
# combine all variables (X's and Y's) into one dataframe
all_variables_reg = pd.concat([X, Y_flourish, Y_depression, Y_anxiety], axis = 1)

In [597]:
# remove all rows with any NaN
print('shape before removing NaNs:', all_variables_reg.shape)
all_variables_reg = all_variables_reg.dropna(axis=0, how='any')
print('shape after removing NaNs:', all_variables_reg.shape) # NOTE: 25,000+ dropped rows were due to AAQ

('shape before removing NaNs:', (45756, 103))
('shape after removing NaNs:', (8972, 103))


In [598]:
# split big dataframe back up
X = all_variables.drop(labels=['flourish', 'deprawsc', 'anx_score'], axis=1)
Y_flourish_reg = all_variables['flourish']
Y_depression_reg = all_variables['deprawsc']
Y_anxiety_reg = all_variables['anx_score']

### 2) Split data into train & test sets

In [599]:
# split the data into training and test sets
X_train_reg, X_test_reg = train_test_split(X, test_size=0.3, random_state=0)
y_flourish_train_reg, y_flourish_test_reg = train_test_split(Y_flourish_reg, test_size=0.3, random_state=0)
y_depression_train_reg, y_depression_test_reg = train_test_split(Y_depression_reg, test_size=0.3, random_state=0)
y_anxiety_train_reg, y_anxiety_test_reg = train_test_split(Y_anxiety_reg, test_size=0.3, random_state=0)

### create and train the classifier

In [606]:
# Create a random forest regressor
clf_reg = RandomForestRegressor(n_estimators=1000, max_features = "auto", max_depth = 5,
                             min_samples_leaf = 100, random_state=0, n_jobs=-1)

In [607]:
# Train the classifier on all three outcomes
clf_reg.fit(X_train_reg, y_depression_train_reg)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=100, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

### extract feature importances

In [608]:
# Print the name and gini importance of each feature
features = pd.DataFrame(sorted(zip(feat_cols, clf_reg.feature_importances_),
                               key=lambda x: x[1], reverse=True), columns = ['name', 'importance'])
features.head(10)

,name,importance
0,AAQ,0.844880
1,aca_impa,0.146528
2,fincur,0.005930
3,finpast,0.000753
4,persist,0.000668
5,educ_par2,0.000232
6,educ_par1,0.000217
7,hours_work_paid,0.000195
8,gpa_sr,0.000100
9,sexual,0.000082


### Test the accuracy of our full-feature classifier

In [613]:
# create RMSE function as absolute measure of fit
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [614]:
# Apply The Full Featured Classifier To The Test Data
y_pred_reg = clf_reg.predict(X_test_reg)

# Measure Goodness-of-Fit
print("R-squared:", clf_reg.score(X_test_reg, y_depression_test_reg))
print("RMSE:", rmse(y_pred_reg, y_depression_test_reg))

('R-squared:', 0.72616560409576358)
('RMSE:', 4.1037553048221866)
